## <center>Лабораторная работа № 5 'Распознавание объектов на фотографиях'<center>

### <center>Выполнила студентка 3 курса группы БФИ2001 Калмыкова Дарья<center>

### Цель
Распознавание объектов на фотографиях (Object Recognition in Photographs)
CIFAR-10 (классификация небольших изображений по десяти классам: самолет, 
автомобиль, птица, кошка, олень, собака, лягушка, лошадь, корабль и грузовик).

### Задачи
* Ознакомиться со сверточными нейронными сетями
* Изучить построение модели в Keras в функциональном виде
* Изучить работу слоя разреживания (Dropout)

### Требования
1. Построить и обучить сверточную нейронную сеть
2. Исследовать работу сети без слоя Dropout
3. Исследовать работу сети при разных размерах ядра свертки

In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data
num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

In [3]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot pencode the labels

In [7]:
# batch_size = 32 # in each iteration, we consider 32 training examples at once
# num_epochs = 200 # we iterate 200 times over the entire training set
# kernel_size = 3 # we will use 3x3 kernels throughout
# pool_size = 2 # we will use 2x2 pooling throughout
# conv_depth_1 = 32 # we will initially have 32 kernels per convolution layer...
# conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
# drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
# drop_prob_2 = 0.5 # dropout in the dense layer with probability 0.5
# hidden_size = 512 # the dense layer will have 512 neurons

In [4]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(32, kernel_size=(3,3), padding='same', 
                       activation='relu')(inp)
conv_2 = Convolution2D(32, kernel_size=(3,3), padding='same', 
                       activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_2)
drop_1 = Dropout(0.25)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(64, kernel_size=(3,3), padding='same', 
                       activation='relu')(drop_1)
conv_4 = Convolution2D(64, kernel_size=(3,3), padding='same',
                       activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_4)
drop_2 = Dropout(0.5)(pool_2)

# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(512, activation='relu')(flat)
drop_3 = Dropout(0.5)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

In [5]:
model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=32, epochs=200,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/200
1407/1407 [==============================] - 127s 89ms/step - loss: 1.5973 - accuracy: 0.4140 - val_loss: 1.1561 - val_accuracy: 0.5904
Epoch 2/200
1407/1407 [==============================] - 128s 91ms/step - loss: 1.2047 - accuracy: 0.5678 - val_loss: 1.0297 - val_accuracy: 0.6454
Epoch 3/200
1407/1407 [==============================] - 125s 89ms/step - loss: 1.0634 - accuracy: 0.6222 - val_loss: 0.8573 - val_accuracy: 0.7040
Epoch 4/200
1407/1407 [==============================] - 129s 92ms/step - loss: 0.9721 - accuracy: 0.6563 - val_loss: 0.8619 - val_accuracy: 0.6978
Epoch 5/200
1407/1407 [==============================] - 126s 90ms/step - loss: 0.9079 - accuracy: 0.6782 - val_loss: 0.7681 - val_accuracy: 0.7410
Epoch 6/200
1407/1407 [==============================] - 126s 89ms/step - loss: 0.8610 - accuracy: 0.6978 - val_loss: 0.7595 - val_accuracy: 0.7406
Epoch 7/200
1407/1407 [==============================] - 126s 89ms/step - loss: 0.8238 - accuracy: 0.7101 - val_

1407/1407 [==============================] - 128s 91ms/step - loss: 0.4609 - accuracy: 0.8403 - val_loss: 0.5721 - val_accuracy: 0.8130
Epoch 57/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.4505 - accuracy: 0.8443 - val_loss: 0.6365 - val_accuracy: 0.7918
Epoch 58/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.4516 - accuracy: 0.8424 - val_loss: 0.5822 - val_accuracy: 0.8100
Epoch 59/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.4437 - accuracy: 0.8456 - val_loss: 0.5836 - val_accuracy: 0.8082
Epoch 60/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.4478 - accuracy: 0.8436 - val_loss: 0.5802 - val_accuracy: 0.8062
Epoch 61/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.4489 - accuracy: 0.8460 - val_loss: 0.5903 - val_accuracy: 0.8070
Epoch 62/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.4494 - accuracy: 0.8440 - val_loss: 

Epoch 111/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3956 - accuracy: 0.8659 - val_loss: 0.6055 - val_accuracy: 0.8066
Epoch 112/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3941 - accuracy: 0.8664 - val_loss: 0.5965 - val_accuracy: 0.8048
Epoch 113/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3927 - accuracy: 0.8670 - val_loss: 0.5919 - val_accuracy: 0.8064
Epoch 114/200
1407/1407 [==============================] - 127s 91ms/step - loss: 0.3818 - accuracy: 0.8691 - val_loss: 0.5887 - val_accuracy: 0.8086
Epoch 115/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3891 - accuracy: 0.8689 - val_loss: 0.5961 - val_accuracy: 0.8064
Epoch 116/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3836 - accuracy: 0.8676 - val_loss: 0.6100 - val_accuracy: 0.8054
Epoch 117/200
1407/1407 [==============================] - 127s 91ms/step - loss: 0.3829 - accuracy:

1407/1407 [==============================] - 128s 91ms/step - loss: 0.3525 - accuracy: 0.8810 - val_loss: 0.6247 - val_accuracy: 0.8118
Epoch 166/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3714 - accuracy: 0.8766 - val_loss: 0.6147 - val_accuracy: 0.7998
Epoch 167/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3625 - accuracy: 0.8795 - val_loss: 0.5936 - val_accuracy: 0.8210
Epoch 168/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3573 - accuracy: 0.8804 - val_loss: 0.5939 - val_accuracy: 0.8194
Epoch 169/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3666 - accuracy: 0.8782 - val_loss: 0.6157 - val_accuracy: 0.8098
Epoch 170/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3597 - accuracy: 0.8804 - val_loss: 0.6383 - val_accuracy: 0.8074
Epoch 171/200
1407/1407 [==============================] - 128s 91ms/step - loss: 0.3613 - accuracy: 0.8818 - val_

[223.86277770996094, 0.5443000197410583]

### Convolution network wo/ Dropout

In [6]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(32, kernel_size=(3,3), padding='same', 
                       activation='relu')(inp)
conv_2 = Convolution2D(32, kernel_size=(3,3), padding='same', 
                       activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_2)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(64, kernel_size=(3,3), padding='same', 
                       activation='relu')(pool_1)
conv_4 = Convolution2D(64, kernel_size=(3,3), padding='same',
                       activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_4)

# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(pool_2)
hidden = Dense(512, activation='relu')(flat)
out = Dense(num_classes, activation='softmax')(hidden)

In [7]:
model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=32, epochs=200,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/200
1407/1407 [==============================] - 126s 89ms/step - loss: 1.3292 - accuracy: 0.5225 - val_loss: 0.9856 - val_accuracy: 0.6542
Epoch 2/200
1407/1407 [==============================] - 126s 90ms/step - loss: 0.8661 - accuracy: 0.6952 - val_loss: 0.7901 - val_accuracy: 0.7264
Epoch 3/200
1407/1407 [==============================] - 126s 90ms/step - loss: 0.6535 - accuracy: 0.7702 - val_loss: 0.7397 - val_accuracy: 0.7468
Epoch 4/200
1407/1407 [==============================] - 127s 90ms/step - loss: 0.4759 - accuracy: 0.8340 - val_loss: 0.7487 - val_accuracy: 0.7510
Epoch 5/200
1407/1407 [==============================] - 119s 85ms/step - loss: 0.3128 - accuracy: 0.8915 - val_loss: 0.8535 - val_accuracy: 0.7536
Epoch 6/200
1407/1407 [==============================] - 118s 84ms/step - loss: 0.1917 - accuracy: 0.9335 - val_loss: 1.0008 - val_accuracy: 0.7558
Epoch 7/200
1407/1407 [==============================] - 118s 84ms/step - loss: 0.1362 - accuracy: 0.9526 - val_

Epoch 56/200
1407/1407 [==============================] - 161s 114ms/step - loss: 0.0420 - accuracy: 0.9901 - val_loss: 3.6450 - val_accuracy: 0.7328
Epoch 57/200
1407/1407 [==============================] - 161s 114ms/step - loss: 0.0571 - accuracy: 0.9884 - val_loss: 3.8962 - val_accuracy: 0.7296
Epoch 58/200
1407/1407 [==============================] - 162s 115ms/step - loss: 0.0436 - accuracy: 0.9907 - val_loss: 3.9641 - val_accuracy: 0.7260
Epoch 59/200
1407/1407 [==============================] - 161s 114ms/step - loss: 0.0398 - accuracy: 0.9910 - val_loss: 3.9118 - val_accuracy: 0.7310
Epoch 60/200
1407/1407 [==============================] - 161s 115ms/step - loss: 0.0439 - accuracy: 0.9900 - val_loss: 4.2126 - val_accuracy: 0.7342
Epoch 61/200
1407/1407 [==============================] - 160s 114ms/step - loss: 0.0480 - accuracy: 0.9900 - val_loss: 3.8015 - val_accuracy: 0.7314
Epoch 62/200
1407/1407 [==============================] - 162s 115ms/step - loss: 0.0402 - accuracy:

Epoch 111/200
1407/1407 [==============================] - 135s 96ms/step - loss: 0.0551 - accuracy: 0.9931 - val_loss: 7.2661 - val_accuracy: 0.7216
Epoch 112/200
1407/1407 [==============================] - 131s 93ms/step - loss: 0.0361 - accuracy: 0.9945 - val_loss: 7.8137 - val_accuracy: 0.7280
Epoch 113/200
1407/1407 [==============================] - 133s 94ms/step - loss: 0.0597 - accuracy: 0.9922 - val_loss: 8.2890 - val_accuracy: 0.7166
Epoch 114/200
1407/1407 [==============================] - 135s 96ms/step - loss: 0.0625 - accuracy: 0.9923 - val_loss: 7.7991 - val_accuracy: 0.7326
Epoch 115/200
1407/1407 [==============================] - 143s 102ms/step - loss: 0.0489 - accuracy: 0.9941 - val_loss: 8.5499 - val_accuracy: 0.7318
Epoch 116/200
1407/1407 [==============================] - 144s 103ms/step - loss: 0.0565 - accuracy: 0.9929 - val_loss: 8.4405 - val_accuracy: 0.7140
Epoch 117/200
1407/1407 [==============================] - 133s 95ms/step - loss: 0.0482 - accurac

1407/1407 [==============================] - 129s 91ms/step - loss: 0.0702 - accuracy: 0.9950 - val_loss: 15.1457 - val_accuracy: 0.7266
Epoch 166/200
1407/1407 [==============================] - 131s 93ms/step - loss: 0.0793 - accuracy: 0.9948 - val_loss: 15.9529 - val_accuracy: 0.7214
Epoch 167/200
1407/1407 [==============================] - 130s 93ms/step - loss: 0.0793 - accuracy: 0.9949 - val_loss: 15.1591 - val_accuracy: 0.7200
Epoch 168/200
1407/1407 [==============================] - 133s 95ms/step - loss: 0.0810 - accuracy: 0.9949 - val_loss: 15.0003 - val_accuracy: 0.7250
Epoch 169/200
1407/1407 [==============================] - 137s 98ms/step - loss: 0.0791 - accuracy: 0.9945 - val_loss: 16.4594 - val_accuracy: 0.7200
Epoch 170/200
1407/1407 [==============================] - 135s 96ms/step - loss: 0.0837 - accuracy: 0.9946 - val_loss: 16.0435 - val_accuracy: 0.7296
Epoch 171/200
1407/1407 [==============================] - 136s 97ms/step - loss: 0.0715 - accuracy: 0.9950 

[13020.2216796875, 0.45669999718666077]

### Trying different kernel_size

In [8]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(32, kernel_size=(4,4), padding='same', 
                       activation='relu')(inp)
conv_2 = Convolution2D(32, kernel_size=(4,4), padding='same', 
                       activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_2)
drop_1 = Dropout(0.25)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(64, kernel_size=(4,4), padding='same', 
                       activation='relu')(drop_1)
conv_4 = Convolution2D(64, kernel_size=(4,4), padding='same',
                       activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(2, 2), strides=2)(conv_4)
drop_2 = Dropout(0.5)(pool_2)

# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(512, activation='relu')(flat)
drop_3 = Dropout(0.5)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

In [9]:
model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=32, epochs=200,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/200
1407/1407 [==============================] - 193s 136ms/step - loss: 1.6482 - accuracy: 0.3898 - val_loss: 1.2587 - val_accuracy: 0.5532
Epoch 2/200
1407/1407 [==============================] - 192s 136ms/step - loss: 1.2896 - accuracy: 0.5374 - val_loss: 1.0527 - val_accuracy: 0.6230
Epoch 3/200
1407/1407 [==============================] - 192s 136ms/step - loss: 1.1448 - accuracy: 0.5899 - val_loss: 0.9646 - val_accuracy: 0.6618
Epoch 4/200
1407/1407 [==============================] - 192s 136ms/step - loss: 1.0627 - accuracy: 0.6238 - val_loss: 0.9122 - val_accuracy: 0.6868
Epoch 5/200
1407/1407 [==============================] - 193s 137ms/step - loss: 0.9984 - accuracy: 0.6451 - val_loss: 0.9098 - val_accuracy: 0.6854
Epoch 6/200
1407/1407 [==============================] - 193s 137ms/step - loss: 0.9581 - accuracy: 0.6628 - val_loss: 0.8386 - val_accuracy: 0.7160
Epoch 7/200
1407/1407 [==============================] - 218s 155ms/step - loss: 0.9185 - accuracy: 0.6751

Epoch 56/200
1407/1407 [==============================] - 189s 134ms/step - loss: 0.5566 - accuracy: 0.8083 - val_loss: 0.6504 - val_accuracy: 0.7902
Epoch 57/200
1407/1407 [==============================] - 189s 135ms/step - loss: 0.5545 - accuracy: 0.8077 - val_loss: 0.6294 - val_accuracy: 0.7986
Epoch 58/200
1407/1407 [==============================] - 189s 134ms/step - loss: 0.5668 - accuracy: 0.8042 - val_loss: 0.6267 - val_accuracy: 0.7984
Epoch 59/200
1407/1407 [==============================] - 189s 134ms/step - loss: 0.5570 - accuracy: 0.8056 - val_loss: 0.6497 - val_accuracy: 0.7906
Epoch 60/200
1407/1407 [==============================] - 189s 134ms/step - loss: 0.5503 - accuracy: 0.8114 - val_loss: 0.6322 - val_accuracy: 0.7940
Epoch 61/200
1407/1407 [==============================] - 190s 135ms/step - loss: 0.5582 - accuracy: 0.8058 - val_loss: 0.6234 - val_accuracy: 0.7964
Epoch 62/200
1407/1407 [==============================] - 189s 135ms/step - loss: 0.5560 - accuracy:

1407/1407 [==============================] - 187s 133ms/step - loss: 0.5030 - accuracy: 0.8300 - val_loss: 0.6473 - val_accuracy: 0.7952
Epoch 111/200
1407/1407 [==============================] - 187s 133ms/step - loss: 0.5031 - accuracy: 0.8310 - val_loss: 0.6268 - val_accuracy: 0.8046
Epoch 112/200
1407/1407 [==============================] - 187s 133ms/step - loss: 0.4987 - accuracy: 0.8314 - val_loss: 0.6378 - val_accuracy: 0.8006
Epoch 113/200
1407/1407 [==============================] - 187s 133ms/step - loss: 0.4950 - accuracy: 0.8330 - val_loss: 0.6240 - val_accuracy: 0.8064
Epoch 114/200
1407/1407 [==============================] - 187s 133ms/step - loss: 0.5003 - accuracy: 0.8316 - val_loss: 0.6324 - val_accuracy: 0.7938
Epoch 115/200
1407/1407 [==============================] - 192s 137ms/step - loss: 0.4925 - accuracy: 0.8343 - val_loss: 0.6559 - val_accuracy: 0.7886
Epoch 116/200
1407/1407 [==============================] - 175s 124ms/step - loss: 0.5022 - accuracy: 0.8297

1407/1407 [==============================] - 188s 134ms/step - loss: 0.4720 - accuracy: 0.8442 - val_loss: 0.6513 - val_accuracy: 0.7960
Epoch 165/200
1407/1407 [==============================] - 187s 133ms/step - loss: 0.4753 - accuracy: 0.8426 - val_loss: 0.6695 - val_accuracy: 0.7860
Epoch 166/200
1407/1407 [==============================] - 187s 133ms/step - loss: 0.4736 - accuracy: 0.8396 - val_loss: 0.6655 - val_accuracy: 0.8022
Epoch 167/200
1407/1407 [==============================] - 188s 134ms/step - loss: 0.4684 - accuracy: 0.8447 - val_loss: 0.6350 - val_accuracy: 0.8046
Epoch 168/200
1407/1407 [==============================] - 188s 134ms/step - loss: 0.4678 - accuracy: 0.8467 - val_loss: 0.6546 - val_accuracy: 0.7992
Epoch 169/200
1407/1407 [==============================] - 188s 133ms/step - loss: 0.4782 - accuracy: 0.8404 - val_loss: 0.6326 - val_accuracy: 0.8020
Epoch 170/200
1407/1407 [==============================] - 188s 134ms/step - loss: 0.4789 - accuracy: 0.8417

[826.74658203125, 0.5241000056266785]